In [1]:
import geopandas as gpd 
import pandas as pd

In [2]:
points = gpd.read_file("../../data/brighton/brighton_points_clean.shp")

In [65]:
import pickle as pkl
with open('../../data/brighton/brighton_graph_sidewalks_only.pkl', 'rb') as pklfile:
    brighton_G = pkl.load(pklfile)

In [4]:
brighton_sidewalks_nodes = brighton_G.nodes(data=True)

In [5]:
import osmnx as ox
from osmnx.distance import great_circle_vec

In [6]:
import networkx as nx
# adapted from osmnx.distance
def is_reachable(id1, id2):
    try:
        nx.shortest_path(brighton_G, id1, id2)
        return True
    except nx.NetworkXNoPath:
        return False

def get_nearest_crosswalk_nodes(G, point, method="haversine", return_dist=False):
    if len(G) < 1:
        raise ValueError("G must contain at least one node")

    # dump graph node coordinates into a pandas dataframe indexed by node id
    # with x and y columns
    coords = ((n, d["x"], d["y"], d["id"]) for n, d in G.nodes(data=True))
    df = pd.DataFrame(coords, columns=["node", "x", "y", "id"]).set_index("node")

    # add columns to df for the (constant) coordinates of reference point
    df["ref_y"] = point[0]
    df["ref_x"] = point[1]

    # calculate the distance between each node and the reference point
    if method == "haversine":
        # calculate distances using haversine for spherical lat-lng geometries
        df['dists'] = great_circle_vec(lat1=df["ref_y"], lng1=df["ref_x"], lat2=df["y"], lng2=df["x"])

    elif method == "euclidean":
        # calculate distances using euclid's formula for projected geometries
        df['dists'] = euclidean_dist_vec(y1=df["ref_y"], x1=df["ref_x"], y2=df["y"], x2=df["x"])

    else:
        raise ValueError('method argument must be either "haversine" or "euclidean"')
    
    six_best = df.nsmallest(n=7,columns=['dists']).iloc[1:]

    vertices = []
    for _, row in six_best.iterrows():
        if len(vertices) > 4 or great_circle_vec(lat1=row.x, lng1=row.ref_y, lat2=row.ref_x, lng2=row.ref_y) > 3:
            continue
        
        reachables = [is_reachable(int(row.id), int(vx.id)) for vx in vertices ]
        if any(reachables):
            continue
        distances = [great_circle_vec(lat1=row.y, lng1=row.x, lat2=vx.y, lng2=vx.x) for vx in vertices]
        
        if all([dist < 4 for dist in distances]):
            vertices.append(row)

    # if caller requested return_dist, return distance between the point and the
    # nearest node as well
    return vertices

In [9]:
print(list(brighton_G.nodes(data=True))[0][1]['geometry'], list(brighton_G.nodes(data=True))[0][1]['x'])

POINT (-71.15143683397585 42.34442187719422) 42.34442187719422


In [10]:
from shapely.geometry import LineString
from math import sqrt
def get_nearest_cw_node_pairs(G, pt):
    pts = get_nearest_crosswalk_nodes(G,pt)
    all_cws = [(u,v) for u in pts for v in pts if u['id'] < v['id']]
    good_cws = sorted(all_cws, key = lambda pair : great_circle_vec(lat1=pair[0].x, lng1=pair[0].y, 
                                                             lat2=pair[1].x, lng2=pair[1].y))
    
    K = len(all_cws)
    num_to_return = int((1 + sqrt(1 + 8*K))/2)
    return good_cws[0:num_to_return]
    
def make_edge(pair):
    u_node = pair[0]
    u = int(u_node.id)
    v_node = pair[1]
    v = int(v_node.id)
    data_dict = {
        'street_id': 0,
        'geometry': LineString([[u_node.y, u_node.x], [v_node.y, v_node.x]]),
        'length_m': great_circle_vec(lat1=u_node.x, lng1=u_node.y, lat2=v_node.x, lng2=v_node.y),
        'angle_deg': 0,
        'osmid': int(u_node.id) * int(v_node.id),
        'angle_class': 0
    }
    return (u,v,data_dict)    

In [11]:
cw_edges = [make_edge(pair) for row in points.iterrows() 
            for pair in get_nearest_cw_node_pairs(brighton_G,(row[1].x, row[1].y))]

In [12]:
len(cw_edges)

1777

In [13]:
len(cw_edges)/len(points)

2.488795518207283

In [ ]:
[edge[2]['length_m'] for edge in cw_edges]

In [66]:
brighton_G.add_edges_from(cw_edges)

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [67]:
def angle_reverse(G):
    rev_edges = nx.reverse(G).edges(data=True)
    def rev_angle(dic):
        dic['angle_deg'] = -dic['angle_deg']
        return dic
    return [(u,v,rev_angle(dat)) for (u,v,dat) in rev_edges]
    
brighton_G.add_edges_from(angle_reverse(brighton_G))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [72]:
from shapely.geometry import Point
import geocoder
g1 = geocoder.osm("23 Claymoss Rd, Boston, MA")
g2 = geocoder.osm("28 Claymoss Rd, Brighton, MA 02135")
#g3 = geocoder.osm("90 monastery rd, Boston, MA")
p1 = Point(g1.json['lng'], g1.json['lat']).coords[0]
p2 = Point(g2.json['lng'], g2.json['lat']).coords[0]
#p3 = Point(g3.json['lng'], g3.json['lat']).coords[0]
n1, d1 = ox.get_nearest_node(brighton_G, p1, method='haversine', return_dist=True)
n2, d2 = ox.get_nearest_node(brighton_G, p2, method='haversine', return_dist=True)
#n3, d3 = ox.get_nearest_node(brighton_G, p3, method='haversine', return_dist=True)
p1, n1, d1, p2, n2, d2


((-71.14476732050123, 42.3440753),
 40279,
 5.883733395563498,
 (-71.14510230000002, 42.34391425),
 85394,
 6.484450046864151)

In [73]:
import networkx as nx

route = nx.shortest_path(brighton_G, n1, n2, weight='length_m')
ox.plot_route_folium(brighton_G, route, route_width = 2)

In [71]:
with open('../../brighton_graph.pkl', 'wb') as pklfile:
    pkl.dump(brighton_G, pklfile)

In [80]:
route

[40279,
 25009,
 25191,
 26888,
 83549,
 41091,
 42849,
 97233,
 72370,
 30166,
 87531,
 10639,
 58561,
 43161,
 94094,
 47026,
 28010,
 83472,
 27465,
 85394]

In [79]:
route_nodes = list(zip(route[:-1], route[1:]))
route_nodes

[(40279, 25009),
 (25009, 25191),
 (25191, 26888),
 (26888, 83549),
 (83549, 41091),
 (41091, 42849),
 (42849, 97233),
 (97233, 72370),
 (72370, 30166),
 (30166, 87531),
 (87531, 10639),
 (10639, 58561),
 (58561, 43161),
 (43161, 94094),
 (94094, 47026),
 (47026, 28010),
 (28010, 83472),
 (83472, 27465),
 (27465, 85394)]

In [81]:
for (u,v) in route_nodes:
    print(u, v, list(brighton_G.edges([u,v])))

40279 25009 [(40279, 3384), (40279, 25009), (25009, 40279), (25009, 25191)]
25009 25191 [(25009, 40279), (25009, 25191), (25191, 25009), (25191, 26888)]
25191 26888 [(25191, 25009), (25191, 26888), (26888, 25191), (26888, 83549)]
26888 83549 [(26888, 25191), (26888, 83549), (83549, 26888), (83549, 41091)]
83549 41091 [(83549, 26888), (83549, 41091), (41091, 83549), (41091, 42849)]
41091 42849 [(41091, 83549), (41091, 42849), (42849, 41091), (42849, 97233)]
42849 97233 [(42849, 41091), (42849, 97233), (97233, 42849), (97233, 72370)]
97233 72370 [(97233, 42849), (97233, 72370), (72370, 97233), (72370, 30166)]
72370 30166 [(72370, 97233), (72370, 30166), (30166, 72370), (30166, 87531)]
30166 87531 [(30166, 72370), (30166, 87531), (87531, 30166), (87531, 10639)]
87531 10639 [(87531, 30166), (87531, 10639), (10639, 87531), (10639, 58561)]
10639 58561 [(10639, 87531), (10639, 58561), (58561, 10639), (58561, 43161)]
58561 43161 [(58561, 10639), (58561, 43161), (43161, 58561), (43161, 94094)]


In [82]:
edge_Getter = brighton_G.edges

In [88]:
edge_Getter[u,v,]

KeyError: 1